In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
import sys, os
sys.path.append(os.path.abspath(".."))
from load_model import *

In [2]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [3]:
# Step 2: Initialize embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

c:\Coding\Gen AI CampusX\genai_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
vectorstore = FAISS.from_documents(docs, embedding_model)

In [5]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [6]:
# Set up the compressor using an LLM
llm = Tiny_llm()
compressor = LLMChainExtractor.from_llm(llm)

c:\Coding\Gen AI CampusX\load_model.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [7]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [8]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [9]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
In this context, "photosynthesis" refers to the process by which green plants convert light energy into chemical energy that they can use for growth and other vital activities. The Grand Canyon is one of the most visited natural wonders in the world, attracting millions of tourists every year who come to see it. The rock formation dates back millions of years, providing insights into Earth's ancient history.

--- Result 2 ---
In medieval Europe, castles were built primarily for defense and were designed with a focus on fortifying the defenses rather than as a means of providing protection from attack. The chlorophyll in plant cells captures sunlight during photosynthesis, which is the process by which plants convert carbon dioxide into oxygen through the enzyme chloroplasts. Knights wore armor made of metal to protect themselves against potential attacks, while siege weapons were often used to breach castle walls.

--- Result 3 ---
In the context of the question "What